###Download Dataset

In [ ]:
!gdown --id '1vrCZjUO0Ht4cl4HtjtcKwjO0GN04MFYm&confirm=t'

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1vrCZjUO0Ht4cl4HtjtcKwjO0GN04MFYm&confirm=t
To: /content/courses.xlsx
100% 15.4k/15.4k [00:00<00:00, 15.9MB/s]


###Install Libraries

In [ ]:
!pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 7.5 MB/s eta 0:00:00


##Import Libraries

In [ ]:
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors

#Load Dataset & Make Some EDA

In [ ]:
df = pd.read_excel('/content/courses.xlsx')
df.head()

,Title,Disc
0,معسكر طويق وعلي بابا للحوسبة السحابية,يقام المعسكر: حضوريًا\nوصف المعسكر: \nيوفّر ا...
1,معسكر تطوير تطبيقات الجوال والويب باستخدام Flu...,يقام المعسكر: حضوريًا \nوصف المعسكر: \nمعسكر ...
2,معسكر Backend باستخدام لغة JAVA,يقام المعسكر: حضوريًا \nوصف المعسكر: \nيمكنك ...
3,معسكر تطوير تطبيقات الهواتف الذكية باستخدام لغ...,يقام المعسكر: حضوريًا \nوصف المعسكر: \nمعسكر ...
4,معسكر تصميم تجربة وواجهات المستخدم,يقام المعسكر: حضوريًا \nوصف المعسكر: \nيستهدف...


In [ ]:
df = df.dropna()
df.head()
# df.isnull()

,Title,Disc
0,معسكر طويق وعلي بابا للحوسبة السحابية,يقام المعسكر: حضوريًا\nوصف المعسكر: \nيوفّر ا...
1,معسكر تطوير تطبيقات الجوال والويب باستخدام Flu...,يقام المعسكر: حضوريًا \nوصف المعسكر: \nمعسكر ...
2,معسكر Backend باستخدام لغة JAVA,يقام المعسكر: حضوريًا \nوصف المعسكر: \nيمكنك ...
3,معسكر تطوير تطبيقات الهواتف الذكية باستخدام لغ...,يقام المعسكر: حضوريًا \nوصف المعسكر: \nمعسكر ...
4,معسكر تصميم تجربة وواجهات المستخدم,يقام المعسكر: حضوريًا \nوصف المعسكر: \nيستهدف...


In [ ]:
df.columns

Index(['Title ', 'Disc'], dtype='object')

In [ ]:
all_desc=df['Disc'].values.tolist()
all_title=df['Title '].values.tolist()

#GPT look

In [ ]:
import requests
import json

url = "https://experimental.willow.vectara.io/v1/chat/completions"

def gpt_look(tit, sen):
    msg = f'"Title: {tit}": \n  Context: {sen}.'
    payload = json.dumps({
      "model": "gpt-3.5-turbo-0301",
      "messages": [{"role": "system", "content": "أنت مساعد مفيد."},
                   {"role": "user","content": msg},
                   {"role": "assistant","content": ' استخرج المعلومات المفيدة من النص المرفق باكبر قدر ممكن من الصراحة ,اجعل اجابتك مختصرة لأقصي حد ممكن, أضف عنوان فريد للنص'},
                   {"role": "assistant","content": 'اختصر النص الناتج' }]
    })
    headers = {
      'Content-Type': 'application/json',
      'customer-id': '348719068',
      'x-api-key': 
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    resp_text = response.text
    dict_ =json.loads(resp_text)
    try:
       ans = dict_["choices"][0]["message"]["content"]
    except:
       return None
    return ans 

In [ ]:
import requests
import json

url = "https://experimental.willow.vectara.io/v1/chat/completions"

def gpt_look(tit, sen):
    msg = f'"Title: {tit}": \n  Context: {sen}.'
    payload = json.dumps({
      "model": "gpt-3.5-turbo-0301",
      "messages": [{"role": "system", "content": "أنت مساعد مفيد."},
                   {"role": "user","content": msg},
                   {"role": "assistant","content": ' استخرج مقطتفات تشير للنص المرفق'},]
    })
    headers = {
      'Content-Type': 'application/json',
      'customer-id': '348719068',
      'x-api-key': 
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    resp_text = response.text
    dict_ =json.loads(resp_text)
    try:
       ans = dict_["choices"][0]["message"]["content"]
    except:
       return None
    return ans 

In [ ]:
import requests
import json

url = "https://experimental.willow.vectara.io/v1/chat/completions"

def gpt_look2(msg):
    payload = json.dumps({
      "model": "gpt-3.5-turbo-0301",
      "messages": [{"role": "system", "content": "أنت مساعد مفيد."},
                   {"role": "user","content": '\n'.join(msg)},
                   {"role": "assistant","content": ' استخرج المعلومات المفيدة من النص المرفق باكبر قدر ممكن من الصراحة ,اجعل اجابتك مختصرة لأقصي حد ممكن, أضف عنوان فريد للنص'},
                   {"role": "assistant","content": 'اختصر النص الناتج' }]
    })
    headers = {
      'Content-Type': 'application/json',
      'customer-id': '348719068',
      'x-api-key': 
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    resp_text = response.text
    dict_ =json.loads(resp_text)
    try:
       ans = dict_["choices"][0]["message"]["content"]
    except:
       return None
    return ans 

In [ ]:
gpt_short = list(map(lambda idx:gpt_look(all_title[idx], all_desc[idx]), range(len(all_desc[:3])))) 

In [ ]:
gpt_short 

['- عنوان النص: معسكر طويق وعلي بابا للحوسبة السحابية.\n- يقام المعسكر حضوريًا ولمدة شهرين من 15 يناير إلى 9 مارس.\n- يهدف المعسكر إلى تدريب المشاركين على المهارات التقنية المهمة للاستفادة من خدمات الحوسبة السحابية.\n- المتطلبات للمشاركة هي الجنسية السعودية وحصول المتقدم على درجة بكالوريوس في إحدى التخصصات التقنية وإجادة اللغة الإنجليزية.\n- بعد إتمام المعسكر، ستحصل على شهادة معتمدة من أكاديمية طويق، بالإضافة إلى التأهيل لشهادة ACA Cloud Computing Certification.',
 '- المعسكر هو معسكر تدريبي مكثف لتطوير تطبيقات الجوال والويب باستخدام إطار عمل Flutter، والذي يعتبر الإطار الأحدث والأسهل لبناء تطبيقات تعمل على عدة أنظمة.\n- ستتعلم في هذا المعسكر على كيفية بناء واجهات التطبيق والتنقل بينها، بالإضافة إلى كيفية التعامل مع البيانات وحفظها.\n- يهدف المعسكر إلى تطوير تطبيقات متقدمة تعمل على نظام iOS ونظام Android وربط التطبيقات مع قواعد بيانات داخلية أو سحابية.\n- تتميز المعسكر باللغة العربية وإمكانية برمجة تطبيق يعمل على جميع المنصات.',
 '- المعسكر هو معسكر Backend باستخدام لغة JAVA.\n- يبدأ ا

In [ ]:
sd = []
for i in range(0, len(gpt_short), 2):
    s = gpt_short[i:i+2]  
    sd.append(gpt_look2(s)) 
    print(i, i+2) 

In [ ]:
gpt_short = sd

In [ ]:
# #Save 
with open('all_short_courses.pkl','wb') as f:
       pickle.dump(gpt_short, f) 


#Load
# with open('all_short_courses.pkl','rb') as f:
#      gpt_short = pickle.load(f)

#Using RoBerta+GPT

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [ ]:
# Sentences we want sentence embeddings for
sentences = all_desc

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-roberta-large-v1')
model = AutoModel.from_pretrained('sentence-transformers/all-roberta-large-v1')

def emb_roberta(sentences):
      # Tokenize sentences
      encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

      # Compute token embeddings
      with torch.no_grad():
          model_output = model(**encoded_input)

      # Perform pooling
      sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

      # Normalize embeddings
      sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
      return sentence_embeddings.cpu().numpy()


In [ ]:
embeddings = emb_roberta(all_desc) #we can run as GPU
neigh = NearestNeighbors(n_neighbors=3)
neigh.fit(embeddings) 

NearestNeighbors(n_neighbors=3)

**Save Embeddings on Drive**

In [ ]:
# #Save 
# with open('all_emb_courses.pkl','wb') as f:
#        pickle.dump(Data_A, f) 


#Load
# with open('all_emb_courses.pkl','rb') as f:
#      Data_A = pickle.load(f)

In [ ]:
import requests
import json

url = "https://experimental.willow.vectara.io/v1/chat/completions"

def gpt3(msg, allsen):
    payload = json.dumps({
      "model": "gpt-3.5-turbo-0301",
      "messages": [{"role": "system", "content": "أنت مساعد مفيد."},
                   {"role": "user","content": msg},
                   {"role": "user","content": f'في حال لم تجد الإجابة في النص السابق, حاول البحث عنه في الفقرة التالية \n {allsen}' },
                   {"role": "assistant","content": 'في حال لم تتمكن من الإجابة حاول مساعدة المستخدم في تحسين السؤال ليناسب النص المقدم'}]
    })
    headers = {
      'Content-Type': 'application/json',
      'customer-id': '348719068',
      'x-api-key': ''
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    resp_text = response.text
    dict_ =json.loads(resp_text)
    try:
       ans = dict_["choices"][0]["message"]["content"]
    except:
       return None
       
    return ans if ans.rfind('UNK') <0 else 'لا أعرف'

In [ ]:
import requests
import json

url = "https://experimental.willow.vectara.io/v1/chat/completions"

def gpt5(msg):
    payload = json.dumps({
      "model": "gpt-3.5-turbo-0301",
      "messages": [{"role": "system", "content": "أنت مساعد مفيد."},
                   {"role": "user","content": msg},
                   {"role": "assistant","content": 'قم بمراعاة اختلاف اللغة وحاول ايجاد الإجابة'},]

    })
    headers = {
      'Content-Type': 'application/json',
      'customer-id': '348719068',
      'x-api-key': 
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    resp_text = response.text
    dict_ =json.loads(resp_text)
    try:
       ans = dict_["choices"][0]["message"]["content"]
    except:
       return None
       
    return ans if ans.rfind('UNK') <0 else None

In [ ]:
import requests
import json

url = "https://experimental.willow.vectara.io/v1/chat/completions"

def gpt_hep(msg, allsen):
    payload = json.dumps({
      "model": "gpt-3.5-turbo-0301",
      "messages": [{"role": "system", "content": "أنت مساعد مفيد."},
                   {"role": "user","content": msg},
                   {"role": "assistant","content": 'في حال لم تتمكن من الإجابة حاول مساعدة المستخدم في تحسين السؤال ليناسب النص المقدم'},
                   {"role": "assistant","content": 'قم بمراعاة اختلاف اللغة وحاول ايجاد الإجابة'}]

    })
    headers = {
      'Content-Type': 'application/json',
      'customer-id': '348719068',
      'x-api-key': 
    }

    return get_ans(requests.request("POST", url, headers=headers, data=payload), allsen)

def get_ans(response, allsen):
    resp_text = response.text
    dict_ =json.loads(resp_text)
    try:
       ans = dict_["choices"][0]["message"]["content"]
    except:
       ans = gpt_hep(make_text(allsen))
       return ans
       
    return ans if ans.rfind('UNK') <0 else gpt_hep(make_text(allsen))

In [ ]:
import requests
import json

url = "https://experimental.willow.vectara.io/v1/chat/completions"

def gpt34(msg):
    payload = json.dumps({
      "model": "gpt-3.5-turbo-0301",
      "messages": [{"role": "system", "content": "أنت مساعد مفيد."},
                   {"role": "user","content": msg},
                   {"role": "assistant","content": 'في حال لم تتمكن من الإجابة حاول مساعدة المستخدم في تحسين السؤال ليناسب النص المقدم'}]
    })
    headers = {
      'Content-Type': 'application/json',
      'customer-id': '348719068',
      'x-api-key': 
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    resp_text = response.text
    dict_ =json.loads(resp_text)
    try:
       ans = dict_["choices"][0]["message"]["content"]
    except:
       return None
       
    return ans if ans.rfind('UNK') <0 else 'لا أعرف'

In [ ]:
import requests
import json

url = "https://experimental.willow.vectara.io/v1/chat/completions"

def comp(tit, sen):
    msg = f'"Title: {tit}": \n  Context: {sen}.'
    payload = json.dumps({
      "model": "gpt-3.5-turbo-0301",
      "messages": [{"role": "system", "content": "أنت مساعد مفيد."},
                   {"role": "user","content": msg},]
    })
    headers = {
      'Content-Type': 'application/json',
      'customer-id': '348719068',
      'x-api-key': 
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    resp_text = response.text
    dict_ =json.loads(resp_text)
    try:
       ans = dict_["choices"][0]["message"]["content"]
    except:
       return None
    return ans 

In [ ]:
# Do this for every question
def make_text(q,context):
    prompt = f"""أجب عن السؤال أدناه بأكبر قدر ممكن من الصدق بإستخدام النص المقدم واذا لم تكن الإجابة مضمنة فقل '[UNK]' 
            Context: {context}
            Q: {q}
            A:"""
    return prompt 

In [ ]:
# Do this for every question
def make_text2(q,context): 
    prompt = f"""أجب عن السؤال أدناه بأكبر قدر من الصدق بإستخدام النص المقدم إذا طلب منك المستخدم إحصاءات ما فقم بالإجابة عنها أو قل '[UNK]' 
            Context: {context}
            Q: {q}
            A:"""
    return prompt 

In [ ]:
# Do this for every question
def make_text3(q,context): 
    prompt = f"""
     Context: {context}
            Q: {q}
            A:"""
    return prompt 

In [ ]:
# Main2 function 
def answer2(text):
    emb = emb_roberta(text)
    scores, indexs = neigh.kneighbors(emb)
    res = '\n'.join([all_desc[i]  for  i in indexs[0]])
    pr= make_text(text,res) 
    ans = gpt5(pr)
    print(ans)
    if ans != 5:
       print("%%%%%%%")
       pr  = make_text2(text,all_title) 
       ans = gpt_hep(pr, all_title)
    print('Q: ', text)
    print('Answer: ')
    print(ans) 

In [ ]:
len(''.join(gpt_short)) 

2216

In [ ]:
answer2('كم عدد المعسكرات') 

لا يوجد معلومات كافية في النص المقدم لتحديد عدد المعسكرات.
%%%%%%%
Q:  كم عدد المعسكرات
Answer: 
يوجد 10 معسكرات وفقاً للنص المقدم.


In [ ]:
answer2('كم عدد المعسكرات المذكورة في النص المقدم بالضبط؟') 

يوجد 3 معسكرات مذكورة في النص المقدم.
%%%%%%%
Q:  كم عدد المعسكرات المذكورة في النص المقدم بالضبط؟
Answer: 
عدد المعسكرات المذكورة في النص المقدم هي عشرة معسكرات.


In [ ]:
answer2('أهداف معسكر اندرويد') 

None
%%%%%%%
Q:  أهداف معسكر اندرويد
Answer: 
من النص المقدم، يمكن ملاحظة وجود معسكر تطوير تطبيقات Android ولكن لم يتم ذكر معلومات حول أهدافه بشكل محدد.


In [ ]:
answer2('ما هي مميزات المعسكر؟') 

Q:  ما هي مميزات المعسكر؟
Answer: 
يمكن الإجابة على سؤالك من خلال النص المقدم لكل معسكر بشكل منفصل. ولكن بشكل عام، تضمن كل معسكر مشاريع عملية وتعليمًا عميقًا في مجالات تقنية حاسوبية محددة، ويتم تقديم الدورات باللغة العربية وحضوريًا. كما يتطلب المعسكر التفرغ للتعلم ونظام كمبيوتر مناسب، ويأتي مع دورات تدريبية في كل من HTML، CSS، ولغة Python. يتيح لك الانتهاء من هذه المهام والتحضير لامتحان الشهادة، والحصول على درجة منقحة من قبل أكاديمية طويق.


In [ ]:
answer2('ما أهداف معسكر علم البيانات؟') 

Q:  ما أهداف معسكر علم البيانات؟
Answer: 
الأهداف الرئيسية لمعسكر علم البيانات هي:

1- العمل مع فريق على مشاريع مختلفة في علم البيانات وتعلم الآلة.
2- معرفة أنواع البيانات وطرق التعامل معها.
3- تحليل البيانات ونمذجتها باستخدام تقنيات التعلم الآلي.


In [ ]:
answer2('ما أهداف مسار علم البيانات في كورسيرا؟') 

Q:  ما أهداف مسار علم البيانات في كورسيرا؟
Answer: 
أنا ذكي ولكني لست قادرًا على الإجابة على هذا السؤال بشكل كامل. يوجد عدة معسكرات في كورسيرا ولكل منها أهداف مختلفة، لذا يرجى تحديد المعسكر المقصود حتى أستطيع الإجابة على السؤال بدقة.
